<h1 style="color: #A5FAB0;font-weight: bold">Parte 1</h1>


Inês deseja abrir uma loja de produtos diversos. Ao ir para o fornecedor Fictício, ela recebeu um menu com diversas produtos e algumas informações relevantes sobre eles. Inês irá primeiro escolher os produtos para depois determinar a quantidade deles a ser comprada

O menu fornecido pela fazenda encontra-se no arquivo "produtos_a.csv"

O arquivo contém as seguintes colunas:

*   Produto: indica o produto a ser comprado
*   Custo_compra: o valor da unidade para comprar do fornecedor Fictício em reais
*   Lucro: o lucro esperado para Inês por unidade do produto em reais
*   Valorização: Uma escala de zero a dez, com precisão de duas casas decimais, que indica a expectativa do lucro aumentar nos próximos meses
*   Demanda: Quantidade do produto comprada no mês anterior
*   Departamento: Qual o departamento do produto sugerido pelo fornecedor Fictício





In [89]:
# Importações
import pandas as pd
import numpy as np
import gurobipy as gp
from gurobipy import GRB

# Leitura do arquivo de dados
df = pd.read_csv('../data/produtos_a.csv')

----

<h1 style="color: #A5FAB0;font-weight: bold">Item 1</h1> 

Auxilie Inês a escolher os melhores produtos elaborando um modelo de otimização que maximize a demanda - assim, espera-se que haja garantia de venda

<h2 style="color: #A5FAB0;font-size: 24px">
a) Defina um modelo para ajudar na tomada de decisão de Inês
</h2> 

Conjuntos\
$F →$ Conjunto de produtos

Parâmetros:\
$I_f →$ Demanda para um produto $f \in F$

Variáveis de decisão:\
$x_f →$ Binária. Indica se um produto $f \in F$ deve ser incluído entre os melhores

Função Objetivo:\
max $\sum_{f \in F} I_fx_f$

<h2 style="color: #A5FAB0;font-size: 24px">
b) Resolva o modelo com um resolvedor
</h2> 

In [90]:
# Iniciado o primeiro modelo
m1 = gp.Model()

# Lista com todos os produtos
F1 = list(df['produto'])

# Lista com as demandas dos produtos
I_f1 = list(df['demanda'])

# Inicializando um vetor com as variáveis binárias de seleção dos produtos
x1 = []
for i in range(len(F1)):
  x1.append(m1.addVar(vtype=GRB.BINARY, name=F1[i]))

# Setando o objetivo do modelo para maximizar a demanda
m1.setObjective(gp.quicksum(x1[i]*I_f1[i] for i in range(len(F1))), GRB.MAXIMIZE)

# Início da impressão de respostas
print('Informações do guroby: ------------------------------------------------- \n')

# Chamando a função de otimização de modelos do Gurobipy
m1.optimize()

# Criando uma lista com os produtos selecionados
produtos1 = []
for i in range(len(F1)):
  if x1[i].x == 1:
    produtos1.append([F1[i], I_f1[i]])

# Criando um dataframe com a lista de produtos selecionados
produtos_df1 = pd.DataFrame(produtos1,columns=['Produto', 'Demanda'])

# Imprimindo dataframe
print('\nDataframe: ------------------------------------------------- \n')
print('Número de produtos: ' + str(len(produtos_df1)))
display(produtos_df1.sort_values(by='Demanda', ascending=False))

# Imprimindo valores ótimos
print('\nInformações sobre resolução: ------------------------------------------------- \n')
print("Valor ótimo: ", m1.objVal)
print("Solução ótima: ", m1.Status == GRB.OPTIMAL)

Informações do guroby: ------------------------------------------------- 

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: Intel(R) Core(TM) i7-9700KF CPU @ 3.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 0 rows, 164 columns and 0 nonzeros
Model fingerprint: 0x6d08d5ef
Variable types: 0 continuous, 164 integer (164 binary)
Coefficient statistics:
  Matrix range     [0e+00, 0e+00]
  Objective range  [6e+02, 5e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [0e+00, 0e+00]
Found heuristic solution: objective 486758.50000

Explored 0 nodes (0 simplex iterations) in 0.00 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 1: 486759 

Optimal solution found (tolerance 1.00e-04)
Best objective 4.867585000000e+05, best bound 4.867585000000e+05, gap 0.0000%

Dataframe: ------------------------------------------------- 

Número de produto

,Produto,Demanda
73,Insecticida,5093.0
113,Quebra Cabeça,5041.9
133,Tesoura,4793.7
24,Carimbo,4766.2
47,Facão,4697.3
...,...,...
53,Gaiola,933.1
118,Relógio,816.9
139,Taco,699.9
154,Varal,585.6



Informações sobre resolução: ------------------------------------------------- 

Valor ótimo:  486758.5
Solução ótima:  True


<h2 style="color: #A5FAB0;font-size: 24px">
c) Discorra sobre a solução obtida
</h2> 

De acordo com a solução ótima realizada pelo Guroby, como não há restrições no modelo, a seleção de produtos que maximiza a demanda é a compra de todos os produtos, totalizando a demanda de> 486.758,5 produtos.

Porém os produtos foram ordenados de descrescente em relação à demanda, para auxiliar Inês em futuras decisões.

----

<h1 style="color: #A5FAB0;font-weight: bold">Item 2</h1> 

Considere que a loja terá somente trinta e cinco tipos de produtos

<h2 style="color: #A5FAB0;font-size: 24px">
a) Altere o modelo para representar esta nova situação
</h2> 

Conjuntos\
$F →$ Conjunto de produtos

Parâmetros:\
$I_f →$ Demanda para um produto $f \in F$

Variáveis de decisão:\
$x_f →$ Binária. Indica se um produto $f \in F$ deve ser incluído entre os melhores

Função Objetivo:\
max $\sum_{f \in F} I_fx_f$

Restrições:\
$\sum_{f \in F} x_f = 35$

<h2 style="color: #A5FAB0;font-size: 24px">
b) Resolva o problema e discorra sobre a solução obtida
</h2> 

In [91]:
# Iniciado o segundo modelo
m2 = gp.Model()

# Lista com todos os produtos
F2 = list(df['produto'])

# Lista com as demandas dos produtos
I_f2 = list(df['demanda'])

# Inicializando um vetor com as variáveis binárias de seleção dos produtos
x2 = []
for i in range(len(F2)):
  x2.append(m2.addVar(vtype=GRB.BINARY, name=F2[i]))

# Setando o objetivo do modelo para maximizar a demanda
m2.setObjective(gp.quicksum(x2[i]*I_f2[i] for i in range(len(F2))), GRB.MAXIMIZE)

# Adicionando uma constraint para que o número de produtos seja igual a 35
m2.addConstr(gp.quicksum(x2[i] for i in range(len(F2))) == 35)

# Início da impressão de respostas
print('Informações do guroby: ------------------------------------------------- \n')

# Chamando a função de otimização de modelos do Gurobipy
m2.optimize()

# Criando uma lista com os produtos selecionados
produtos2 = []
for i in range(len(F2)):
  if x2[i].x == 1:
    produtos2.append([F2[i], I_f2[i]])

# Criando um dataframe com a lista de produtos selecionados
produtos_df2 = pd.DataFrame(produtos2,columns=['Produto', 'Demanda'])

# Imprimindo dataframe
print('\nDataframe: ------------------------------------------------- \n')
print('Número de produtos: ' + str(len(produtos_df2)))
display(produtos_df2.sort_values(by='Demanda', ascending=False))

# Imprimindo valores ótimos
print('\nInformações sobre resolução: ------------------------------------------------- \n')
print("Valor ótimo: ", m2.objVal)
print("Solução ótima: ", m2.Status == GRB.OPTIMAL)

Informações do guroby: ------------------------------------------------- 

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: Intel(R) Core(TM) i7-9700KF CPU @ 3.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 1 rows, 164 columns and 164 nonzeros
Model fingerprint: 0xd390ee4c
Variable types: 0 continuous, 164 integer (164 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [6e+02, 5e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [4e+01, 4e+01]
Found heuristic solution: objective 97099.400000
Presolve removed 1 rows and 164 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 2: 149805 97099.4 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.498054000000e+05, best bound 1.4

,Produto,Demanda
12,Insecticida,5093.0
22,Quebra Cabeça,5041.9
27,Tesoura,4793.7
5,Carimbo,4766.2
8,Facão,4697.3
14,Lampada,4650.6
33,Vela,4581.8
32,Umidificador de Ar,4509.3
34,Xicara,4498.1
6,Espátula,4479.1



Informações sobre resolução: ------------------------------------------------- 

Valor ótimo:  149805.4
Solução ótima:  True


Adicionando a restrição de até 35 produtos, o gurobipy chegou a uma solução ótima que maximiza a demanda, chegando ao valor de 149.
805,4 itens.

----

<h1 style="color: #A5FAB0;font-weight: bold">Item 3</h1> 

Devido a questões logística, o fornecedor possui somente produtos de demanda no intervalo [1100, 3800]

<h2 style="color: #A5FAB0;font-size: 24px">
a) Altere o modelo para representar esta nova situação
</h2> 

Conjuntos\
$F →$ Conjunto de produtos

Parâmetros:\
$I_f →$ Demanda para um produto $f \in F$

Variáveis de decisão:\
$x_f →$ Binária. Indica se um produto $f \in F$ deve ser incluído entre os melhores

Função Objetivo:\
max $\sum_{f \in F} I_fx_f$

Restrições:\
$x_f*I_f <= 3800 →$ Produtos devem ter demanda menor ou igual a 3800

$x_f*I_f >= 1100 * x_f →$ Produtos devem ter demanda maior ou igual a 1100 (apenas caso sejam selecionados pelo modelo)

<h2 style="color: #A5FAB0;font-size: 24px">
b) Resolva o problema e discorra sobre a solução obtida
</h2> 

In [92]:
# Iniciado o terceiro modelo
m3 = gp.Model()

# Lista com todos os produtos
F3 = list(df['produto'])

# Lista com as demandas dos produtos
I_f3 = list(df['demanda'])

# Inicializando um vetor com as variáveis binárias de seleção dos produtos
x3 = []
for i in range(len(F3)):
  x3.append(m3.addVar(vtype=GRB.BINARY, name=F3[i]))

# Setando o objetivo do modelo para maximizar a demanda
m3.setObjective(gp.quicksum(x3[i]*I_f3[i] for i in range(len(F3))), GRB.MAXIMIZE)

# Adicionando uma constraint para que a demanda dos produtos selecionados estejam entre 1100 e 3800
for i in range(len(F3)):
    m3.addConstr(x3[i] * I_f3[i] <= 3800)
    m3.addConstr(x3[i] * I_f3[i] >= 1100 * x3[i])

# Início da impressão de respostas
print('Informações do guroby: ------------------------------------------------- \n')

# Chamando a função de otimização de modelos do Gurobipy
m3.optimize()

# Criando uma lista com os produtos selecionados
produtos3 = []
for i in range(len(F3)):
  if x3[i].x == 1:
    produtos3.append([F3[i], I_f3[i]])

# Criando um dataframe com a lista de produtos selecionados
produtos_df3 = pd.DataFrame(produtos3,columns=['Produto', 'Demanda'])

# Imprimindo dataframe
print('\nDataframe: ------------------------------------------------- \n')
print('Número de produtos: ' + str(len(produtos_df3)))
display(produtos_df3.sort_values(by='Demanda', ascending=False))

# Imprimindo valores ótimos
print('\nInformações sobre resolução: ------------------------------------------------- \n')
print("Valor ótimo: ", m3.objVal)
print("Solução ótima: ", m3.Status == GRB.OPTIMAL)

Informações do guroby: ------------------------------------------------- 

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: Intel(R) Core(TM) i7-9700KF CPU @ 3.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 328 rows, 164 columns and 328 nonzeros
Model fingerprint: 0xa7876fc3
Variable types: 0 continuous, 164 integer (164 binary)
Coefficient statistics:
  Matrix range     [4e+01, 5e+03]
  Objective range  [6e+02, 5e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [4e+03, 4e+03]
Found heuristic solution: objective 337115.50000
Presolve removed 328 rows and 164 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 1: 337116 

Optimal solution found (tolerance 1.00e-04)
Best objective 3.371155000000e+05, best bound 3.37115

,Produto,Demanda
36,Espelho,3773.5
72,Lata,3772.1
64,Lápis,3721.7
105,Telefone,3710.0
41,Funil,3696.7
...,...,...
17,Copo,1451.7
55,Imã,1431.5
123,Xarope,1329.9
77,Navio de Brinquedo,1287.4



Informações sobre resolução: ------------------------------------------------- 

Valor ótimo:  337115.5
Solução ótima:  True


Para a remoção dos produtos com demanda inferior a 1100 e superior a 3800 foi realizada uma operação condicional sobre o dataframe. Após a remoção das linhas que não eram desejada, o mesmo modelo anterior foi realizado novamente, totalizando uma demanda de 249.242,79 itens e se utilizando de 125 produtos diferentes. Essa é uma solução ótima.

----

<h1 style="color: #A5FAB0;font-weight: bold">Item 4</h1> 

Devido a seu interesse em expandir seus lucros, Inês decidiu escolher produtos de acordo com a sua possibilidade de valorização, ao invés da demanda

<h2 style="color: #A5FAB0;font-size: 24px">
a) Altere o modelo para representar esta nova situação
</h2> 

Conjuntos\
$F →$ Conjunto de produtos

Parâmetros:\
$I_f →$ Valorização para um produto $f \in F$

Variáveis de decisão:\
$x_f →$ Binária. Indica se um produto $f \in F$ deve ser incluído entre os melhores

Função Objetivo:\
max $\sum_{f \in F} I_fx_f$

<h2 style="color: #A5FAB0;font-size: 24px">
b) Resolva o problema e discorra sobre a solução obtida
</h2> 

In [93]:
# Iniciado o quarto modelo
m4 = gp.Model()

# Lista com todos os produtos
F4 = list(df['produto'])

# Lista com as valorizações dos produtos
I_f4 = list(df['valorizacao'])

# Inicializando um vetor com as variáveis binárias de seleção dos produtos
x4 = []
for i in range(len(F4)):
  x4.append(m4.addVar(vtype=GRB.BINARY, name=F4[i]))

# Setando o objetivo do modelo para maximizar a valorização
m4.setObjective(gp.quicksum(x4[i]*I_f4[i] for i in range(len(F4))), GRB.MAXIMIZE)

# Início da impressão de respostas
print('Informações do guroby: ------------------------------------------------- \n')

# Chamando a função de otimização de modelos do Gurobipy
m4.optimize()

# Criando uma lista com os produtos selecionados
produtos4 = []
for i in range(len(F4)):
  if x4[i].x == 1:
    produtos4.append([F4[i], I_f4[i]])

# Criando um dataframe com a lista de produtos selecionados
produtos_df4 = pd.DataFrame(produtos4,columns=['Produto', 'Valorizacao'])

# Imprimindo dataframe
print('\nDataframe: ------------------------------------------------- \n')
print('Número de produtos: ' + str(len(produtos_df4)))
display(produtos_df4.sort_values(by='Valorizacao', ascending=False))

# Imprimindo valores ótimos
print('\nInformações sobre resolução: ------------------------------------------------- \n')
print("Valor ótimo: ", m4.objVal)
print("Solução ótima: ", m4.Status == GRB.OPTIMAL)

Informações do guroby: ------------------------------------------------- 

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: Intel(R) Core(TM) i7-9700KF CPU @ 3.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 0 rows, 164 columns and 0 nonzeros
Model fingerprint: 0xc8443fd8
Variable types: 0 continuous, 164 integer (164 binary)
Coefficient statistics:
  Matrix range     [0e+00, 0e+00]
  Objective range  [2e-01, 9e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [0e+00, 0e+00]
Found heuristic solution: objective 720.8000000

Explored 0 nodes (0 simplex iterations) in 0.00 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 1: 720.8 

Optimal solution found (tolerance 1.00e-04)
Best objective 7.208000000000e+02, best bound 7.208000000000e+02, gap 0.0000%

Dataframe: ------------------------------------------------- 

Número de produtos:

,Produto,Valorizacao
110,Porcelanato,9.46
118,Relógio,9.30
105,Papel,8.75
65,Hélice,8.75
23,Copo,8.68
...,...,...
112,Queijeira,1.10
52,Farol,0.79
15,CD,0.79
133,Tesoura,0.29



Informações sobre resolução: ------------------------------------------------- 

Valor ótimo:  720.8
Solução ótima:  True


Como não há restrições no modelo, todos os produtos são parte da resolução que otimiza a valorização dos produtos, no output, esses produtos foram organizados de maneira descrescente, para que os mais interessantes sejam visualizados primeiro. O valor maximizado de valorização é de 720,8, que é um valor ótimo.

----

<h1 style="color: #A5FAB0;font-weight: bold">Item 5</h1> 

Haja vista que a demanda não é o único fator determinante para o lucro, Inês decidiu que a demanda total dos produtos não deve ser superior a 80000

<h2 style="color: #A5FAB0;font-size: 24px">
a) Altere o modelo para representar esta nova situação
</h2> 

Conjuntos\
$F →$ Conjunto de produtos

Parâmetros:\
$I_f →$ Demanda para um produto $f \in F$

Variáveis de decisão:\
$x_f →$ Binária. Indica se um produto $f \in F$ deve ser incluído entre os melhores

Função Objetivo:\
max $\sum_{f \in F} I_fx_f$

Restrições:\
$\sum_{x_f*I_f} <= 80.000 →$ Produtos total dos produtos não deve ser maior que 80.000

<h2 style="color: #A5FAB0;font-size: 24px">
b) Resolva o problema e discorra sobre a solução obtida
</h2> 

In [94]:
# Iniciado o quinto modelo
m5 = gp.Model()

# Lista com todos os produtos
F5 = list(df['produto'])

# Lista com as demandas dos produtos
I_f5 = list(df['demanda'])

# Inicializando um vetor com as variáveis binárias de seleção dos produtos
x5 = []
for i in range(len(F5)):
  x5.append(m5.addVar(vtype=GRB.BINARY, name=F5[i]))

# Setando o objetivo do modelo para maximizar a demanda
m5.setObjective(gp.quicksum(x5[i]*I_f5[i] for i in range(len(F5))), GRB.MAXIMIZE)

# Adicionando uma constraint para que a demanda total dos produtos selecionados não sejam superiores a 80.000
m5.addConstr(gp.quicksum(x5[i]*I_f5[i] for i in range(len(F5))) <= 80000 )

# Início da impressão de respostas
print('Informações do guroby: ------------------------------------------------- \n')

# Chamando a função de otimização de modelos do Gurobipy
m5.optimize()

# Criando uma lista com os produtos selecionados
produtos5 = []
for i in range(len(F5)):
  if x5[i].x == 1:
    produtos5.append([F5[i], I_f5[i]])

# Criando um dataframe com a lista de produtos selecionados
produtos_df5 = pd.DataFrame(produtos5,columns=['Produto', 'Demanda'])

# Imprimindo dataframe
print('\nDataframe: ------------------------------------------------- \n')
print('Número de produtos: ' + str(len(produtos_df5)))
display(produtos_df5.sort_values(by='Demanda', ascending=False))

# Imprimindo valores ótimos
print('\nInformações sobre resolução: ------------------------------------------------- \n')
print("Valor ótimo: ", m5.objVal)
print("Solução ótima: ", m5.Status == GRB.OPTIMAL)

Informações do guroby: ------------------------------------------------- 

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: Intel(R) Core(TM) i7-9700KF CPU @ 3.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 1 rows, 164 columns and 164 nonzeros
Model fingerprint: 0xad10df26
Variable types: 0 continuous, 164 integer (164 binary)
Coefficient statistics:
  Matrix range     [6e+02, 5e+03]
  Objective range  [6e+02, 5e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [8e+04, 8e+04]
Found heuristic solution: objective 79718.600000
Presolve time: 0.00s
Presolved: 1 rows, 164 columns, 164 nonzeros
Variable types: 0 continuous, 164 integer (164 binary)
Found heuristic solution: objective 79998.700000

Root relaxation: objective 8.000000e+04, 5 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj 

,Produto,Demanda
18,Carimbo,4766.2
3,Aro,4343.4
15,Camisola,4213.0
14,Camiseta,4194.9
13,Camisa,4138.0
6,Abajour,3948.9
12,Caneta,3306.1
25,Garrafa,3231.2
27,Lapiseira,3213.8
9,Borracha,3022.0



Informações sobre resolução: ------------------------------------------------- 

Valor ótimo:  79998.69999999998
Solução ótima:  True


Como existe a restrição de demanda máxima sendo 80000 unidades, então o gurobipy tem como objetivo fazer o melhor mix de produtos para que a somatória total da demanda chegue o maix próximo possível de 80000, sem exceder. Para isso, foram utilizados 29 produtos listados acima, totalizando uma demanda de 79.998,69 items, que é uma solução ótima.

----

<h1 style="color: #A5FAB0;font-weight: bold">Item 6</h1> 

A partir do modelo elaborado, vamos explorar alguns parâmetros do resolvedor

<h2 style="color: #A5FAB0;font-size: 24px">
a) Pesquise e explique o papel dos parâmetros "PoolSearchMode" e "PoolSolutions"
</h2> 

O parâmetro Pool Search Mode é utilizado para selecionar diferentes modeos de explorar o árvore de busca do solver. Por padrão, esse parâmetro é setada como 0, então o solver busca uma solução ótima para o modelo. Ao setar esse parâmetro para um valor não-padrão, ele passa a continuar buscar por outras respostas de alta qualidade, mesmo após ter envontrado uma solução ótima.
Os valores para esse parâmetro podem ser: 0 para funcionamento 'padrão', 1 para funcionamento 'basic', onde soluções intermediárias são armazenadas, porém não utilizadas para melhorar o limite superior da otimização, e 2 para funcionamento 'alternate', onde suluções intermediárias são salvas e utilizadas no limite superior da otimização.

Já o parâmetro PoolSolutions é uma parâmetro utilizado para definir quantas soluções para um modelo o solver deve obter, por padrão, o solver busca apena por soluções ótimas.

<h2 style="color: #A5FAB0;font-size: 24px">
b) Indique se há mais de uma solução ótima para o problema
</h2> 

In [95]:
# Iniciado o sexto modelo
m6 = gp.Model()

# Setando os parâmetros 'PoolSearchMode' e 'PoolSolutions'
m6.setParam('PoolSearchMode', 2)
m6.setParam('PoolSolutions', 10)

# Lista com todos os produtos
F6 = list(df['produto'])

# Lista com as demandas dos produtos
I_f6 = list(df['demanda'])

# Inicializando um vetor com as variáveis binárias de seleção dos produtos
x6 = []
for i in range(len(F6)):
  x6.append(m6.addVar(vtype=GRB.BINARY, name=F6[i]))

# Setando o objetivo do modelo para maximizar a demanda
m6.setObjective(gp.quicksum(x6[i]*I_f6[i] for i in range(len(F6))), GRB.MAXIMIZE)

# Adicionando uma constraint para que a demanda total dos produtos selecionados não sejam superiores a 80.000
m6.addConstr(gp.quicksum(x6[i]*I_f6[i] for i in range(len(F6))) <= 80000 )

# Início da impressão de respostas
print('\nInformações do guroby: ------------------------------------------------- \n')

# Chamando a função de otimização de modelos do Gurobipy
m6.optimize()

Set parameter PoolSearchMode to value 2

Informações do guroby: ------------------------------------------------- 

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: Intel(R) Core(TM) i7-9700KF CPU @ 3.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 1 rows, 164 columns and 164 nonzeros
Model fingerprint: 0xad10df26
Variable types: 0 continuous, 164 integer (164 binary)
Coefficient statistics:
  Matrix range     [6e+02, 5e+03]
  Objective range  [6e+02, 5e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [8e+04, 8e+04]
Found heuristic solution: objective 79718.600000
Presolve time: 0.00s
Presolved: 1 rows, 164 columns, 164 nonzeros
Variable types: 0 continuous, 164 integer (164 binary)
Found heuristic solution: objective 79998.700000

Root relaxation: objective 8.000000e+04, 5 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bo

In [96]:
# Criando uma lista com os produtos selecionados
produtos6 = []
for i in range(len(F6)):
  if x6[i].x == 1:
    produtos6.append([F6[i], I_f6[i]])

# Criando um dataframe com a lista de produtos selecionados
produtos_df6 = pd.DataFrame(produtos6,columns=['Produto', 'Demanda'])

# Imprimindo dataframe
print('\nDataframe: ------------------------------------------------- \n')
print('Número de produtos: ' + str(len(produtos_df6)))
display(produtos_df6.sort_values(by='Demanda', ascending=False))

# Imprimindo valores ótimos
print('\nInformações sobre resolução: ------------------------------------------------- \n')
print("Valor ótimo: ", m6.objVal)
print("Solução ótima: ", m6.Status == GRB.OPTIMAL)


Dataframe: ------------------------------------------------- 

Número de produtos: 30


,Produto,Demanda
15,Lampada,4650.6
25,Umidificador de Ar,4509.3
21,Revista,4377.4
9,Gaita,4184.4
23,Telefone,3710.0
2,Dedal,3645.1
28,Xale,3273.3
16,Luminária,3205.8
14,Janela,3188.6
6,Espelho,3000.5



Informações sobre resolução: ------------------------------------------------- 

Valor ótimo:  79999.99999999999
Solução ótima:  True


De acordo com o Gurobipy, há mais de uma solução ótima para o problema. Foram encontradas 10 soluções para o problema, e, aparentemente o número de produtos selecionados aumentou de 29 para 30.

----

<h1 style="color: #A5FAB0;font-weight: bold">Item 7</h1> 

Façamos um último acréscimo ao modelo: Inês deseja garantir um lucro mínimo por meio da venda de produtos. Para isso, ela deseja que cada um dos produtos escolhidos tenham um lucro de, pelo menos, cento e cinquenta reais

<h2 style="color: #A5FAB0;font-size: 24px">
a) Altere o modelo para representar esta nova situação
</h2> 

Conjuntos\
$F →$ Conjunto de produtos

Parâmetros:\
$I_f →$ Lucro para um produto $f \in F$

Variáveis de decisão:\
$x_f →$ Binária. Indica se um produto $f \in F$ deve ser incluído entre os melhores

Função Objetivo:\
max $\sum_{f \in F} I_fx_f$

Restrições:\
$x_f*I_f >= 150 →$ Lucro dos produtos deve ser maior que 150

<h2 style="color: #A5FAB0;font-size: 24px">
b) Resolva o problema e discorra sobre a solução obtida
</h2> 

In [97]:
# Iniciado o sétimo modelo
m7 = gp.Model()

# Lista com todos os produtos
F7 = list(df['produto'])

# Lista com os lucros individuais dos produtos
I_f7 = list(df['lucro'])

# Inicializando um vetor com as variáveis binárias de seleção dos produtos
x7 = []
for i in range(len(F7)):
  x7.append(m7.addVar(vtype=GRB.BINARY, name=F7[i]))

# Setando o objetivo do modelo para maximizar o lucro
m7.setObjective(gp.quicksum(x7[i]*I_f7[i] for i in range(len(F7))), GRB.MAXIMIZE)

# Adicionando uma constraint para que o lucro mínimo de cada produto seja maior ou igual a 150
for i in range(len(F7)):
    m7.addConstr(x7[i] * I_f7[i] >= 150 * x7[i])

# Início da impressão de respostas
print('\nInformações do guroby: ------------------------------------------------- \n')

# Chamando a função de otimização de modelos do Gurobipy
m7.optimize()

# Criando uma lista com os produtos selecionados
produtos7 = []
for i in range(len(F7)):
  if x7[i].x == 1:
    produtos7.append([F7[i], I_f7[i]])

# Criando um dataframe com a lista de produtos selecionados
produtos_df7 = pd.DataFrame(produtos7,columns=['Produto', 'Lucro'])

# Imprimindo dataframe
print('\nDataframe: ------------------------------------------------- \n')
print('Número de produtos: ' + str(len(produtos_df7)))
display(produtos_df7.sort_values(by='Lucro', ascending=False))

# Imprimindo valores ótimos
print('\nInformações sobre resolução: ------------------------------------------------- \n')
print("Valor ótimo: ", m7.objVal)
print("Solução ótima: ", m7.Status == GRB.OPTIMAL)


Informações do guroby: ------------------------------------------------- 

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: Intel(R) Core(TM) i7-9700KF CPU @ 3.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 164 rows, 164 columns and 164 nonzeros
Model fingerprint: 0xe0d5787b
Variable types: 0 continuous, 164 integer (164 binary)
Coefficient statistics:
  Matrix range     [8e-02, 1e+02]
  Objective range  [1e+01, 2e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [0e+00, 0e+00]
Found heuristic solution: objective 7413.1500000
Presolve removed 164 rows and 164 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.00 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 1: 7413.15 

Optimal solution found (tolerance 1.00e-04)
Best objective 7.413150000000e+03, best bound 7.413

,Produto,Lucro
31,Termômetro,199.77
38,Vara de pescar,199.38
24,Roleta,198.55
37,Urna,196.22
29,Sacola,196.17
13,Jade,195.45
34,Talismã,195.27
11,Incenso,194.82
1,Almofada,194.19
20,Prato,192.99



Informações sobre resolução: ------------------------------------------------- 

Valor ótimo:  7413.150000000001
Solução ótima:  True


Nesse modelo, o guroby otimizou o lucro selecionando 42 produtos, totalizando 7413 de somatória de lucro individual dos produtos selecionados. O valor obtido é ótimo.

----

<h1 style="color: #A5FAB0;font-weight: bold">Item 8</h1> 

A partir dos problemas desenvolvidos nesta questão responda:

<h2 style="color: #A5FAB0;font-size: 24px">
a) Como o acréscimo de restrições interfere na solução ótima do problema?
</h2> 

O acréscimo de restrições costuma alterar os produtos selecionados, porém os modelos sempre obtém rápidamente uma solução ótima para o problema.
Além disso, a depender das restrições incluidas, o tempo de resolução do GHuroby muda.

Ao utilizar os parâmetros citados no item 6, podemos ver que existem soluções ótimas alternativas.